<h1>Transfer Learning</h1>

In diesem Abschnitt wird erklärt, was Transfer-Learning ist und was es damit Aufsicht hat.

Es ist einfach in wenigen Worten erklärt. <br>
Wir nehmen ein Model, das für ein Problem trainiert wurde und nutzen es um ein ähnliches Problem zu lösen.

Ein Model, das z. B. mit Millionen von Daten trainiert wurde, um verschiedene Objekte, Klasse, usw., zu erkennen, erzielt gute Performance auf solchen Daten. Das Model kann z. B. Tiere und Früchte erkenne, dabei hat es aber nie eine Birne gesehen.

Wir nehmen das Model und trainieren nur die letzte (oder paar letzten) Layers, um es an unser Problem anzupassen. <br>
=> Das Ereknnen einer Birne.

Eine Birne hat ähnliche Merkmale wie ein Apfel, was z. B. von dem Pre-Trained Model genutzt wurde. Mit ein paar Trainingsschritten kann es dann auch einen Apfel erkenne.

Ein anderes Beispiel wäre, das Model wurde mit vielen Daten darauf trainiert, PKWs zu erkennen. Dasselbe Model kann dazu benutzt werden, LKWs zu erkennen, mit ein paar Änderungen des letzten Layers.
- Beim Trainieren sind die Weights der anderen Layers statisch und werden sich nicht ändern.
- Der Vorteil ist, dass bei dieser Art des Trainings weniger Rechenleistung und Zeit nötig ist, um die richtigen Weights des Models zu finden.

Hier wird ein Model aus dem Tensorflow Hub geladen und darauf trainiert Blumen zu erkennen.

In [3]:
# // Conten Coming

In [9]:
# Imports.
import tensorflow as tf
import numpy as np
import PIL
import cv2   
import pathlib
import tensorflow_hub as hub
import pathlib

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
# Hole Dataset
# - Siehe Tensorflow: https://www.tensorflow.org/datasets/catalog/tf_flowers
url = "http://download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=url, cache_dir='./data/datasets/tf_flowers', untar=True)

In [37]:
# https://www.tensorflow.org/hub
# - Model: https://www.kaggle.com/models/google/mobilenet-v2   "mobilenet_v2"
# - Füge noch dritte Dimension ein, wegen dem Dataset.  

# mobilenet_v2/classification/4 => Normales Model
# Wir Brauchen ein Model ohne das letzte Layer.
#    mobilenet_v2 ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
mobilenet_v2 ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"  # feature_vector => Model ohne letzte Layer.

# Lade das Model so:
# - trainable: Weights sind statisch. 
mobile = hub.KerasLayer(mobilenet_v2, input_shape=(224,224)+(3,), trainable=False)

#   mobile = tf.keras.applications.MobileNetV2(input_shape=(224,224)+(3,), )
# - Im Hub sind noch andere gute Modelle. 

In [34]:
# Zeige Zusammenfassung des Models. 
#mobile.summary() # Nur bei Modellen wie  mobilenet_v2 ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"

In [12]:
# Das Beispiel mit den Blumen ist aus dem Notebook Data-Argumentation.

url      = "http://download.tensorflow.org/example_images/flower_photos.tgz"  # Dataset.
data_dir = pathlib.Path(data_dir)                                   # Dataset Pfad. 
flowers  = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']  # Klassen

# Erstelle Pfade.
list_flowers = {}
for i in flowers:
    list_flowers[i] = list(data_dir.glob(f"{i}/*.jpg"))

In [13]:
list_flowers.keys()

dict_keys(['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips'])

In [16]:
list_flowers['daisy'][0]

WindowsPath('data/datasets/tf_flowers/datasets/flower_photos/daisy/100080576_f52e8ee070_n.jpg')

Wenn alle Pfade gesetzt sind, können wir das Dataset mit X, y erstellen. <br>
Dabei muss die Größer der Bilder geändert werden, dann die Bilder normalisieren, und zum Schluss die Labels codieren. 
- Achte dabei das die Bilder die richtige Größe haben. 

In [25]:
# Erstelle Dataset.:
X_data = [] # Blumen.
y_data = [] # Labels. 

# Lese Bilder und ändere Größe.
for key, items in list_flowers.items():

    for item in items:
        img = cv2.resize( cv2.imread(item), (224, 224))
        X_data.append(img)
        y_data.append(key)

name_dict = {'daisy':0, 'dandelion':1, 'roses':2, 'sunflowers':3, 'tulips':4 }
y_data_new = [name_dict[item]  for item in y_data]  # Liste mit Labels. 

# Als Numpy.
X_data = np.array(X_data)
del y_data
y_data = np.array(y_data_new)
del y_data_new

In [26]:
X_data[0].shape

(224, 224, 3)

Teile wie üblich Dataset in Train- und Testset auf.

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=.2 )

In [28]:
# Normalisiere
X_train_sc = X_train / 255
X_test_sc  = X_test / 255

Jetzt haben wir die Layers des Models und können es für das Training nutzt. 

In [55]:
# Erstelle Model

# Wrapper- damit die Klasse übereinstimmt.
# - tensorflow_hub.keras_layer.KerasLayer

class CustomHubLayer(tf.keras.layers.Layer):
    def __init__(self, hub_url, input_shape, trainable=False):
        super(CustomHubLayer, self).__init__()
        self.hub_layer = hub.KerasLayer(hub_url, input_shape=input_shape, trainable=trainable)
    
    def call(self, inputs):
        return self.hub_layer(inputs)

model_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
mobile    = CustomHubLayer(model_url, (244,244,3), trainable=False)

my_model = tf.keras.Sequential([
    mobile, # Enthählt bereits alle Layer die wir brauchen.
    # Füge Layers, etc., hinzu.
    tf.keras.layers.Dense(5)  # 5 Klassen
])

my_model.compile(
    optimizer = 'adam',
    loss      = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics   = ['accuracy']
)

In [56]:
# Trainiere Model. 
my_model.fit(X_train_sc, y_train, epochs=3)

Epoch 1/3



92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 191ms/step - accuracy: 0.5882 - loss: 1.0451
Epoch 2/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 18s 196ms/step - accuracy: 0.8638 - loss: 0.4075
Epoch 3/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 19s 202ms/step - accuracy: 0.8957 - loss: 0.3282


Nach nur 3 Epochen erzielt das Model beim Trainieren eine bessere Genauigkeit als ein Model, deren Weights nicht angepasst wurden. Mit dieser Methode kann Zeit und Rechenleistung gespart werden.

In [57]:
my_model.evaluate(X_test_sc, y_test)

23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 198ms/step - accuracy: 0.8561 - loss: 0.3660


[0.359163761138916, 0.8610354065895081]

Achte darauf, dass die Klassen beim Erstellen eines sequenziellen Models übereinstimmen. <br>
Sonst müssen vorher noch einige Schritte durchlaufen werden.